In [1]:
import pandas as pd
import numpy as np

In [2]:
labels = pd.read_csv("labels.csv")
print(labels.shape)
print(labels.head())

(10222, 2)
                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [3]:
targets = labels["breed"]
print(targets.head())
filenames = labels["id"]
print(filenames.head())

0         boston_bull
1               dingo
2            pekinese
3            bluetick
4    golden_retriever
Name: breed, dtype: object
0    000bec180eb18c7604dcecc8fe0dba07
1    001513dfcb2ffafc82cccf4d8bbaba97
2    001cdf01b096e06d78e9e5112d419397
3    00214f311d5d2247d5dfe4fe24b2303d
4    0021f9ceb3235effd7fcde7f7538ed62
Name: id, dtype: object


In [4]:
values = np.array(targets)
indices = np.array(filenames)
label_df = pd.DataFrame(values,index = indices)
print(label_df.head())

                                                 0
000bec180eb18c7604dcecc8fe0dba07       boston_bull
001513dfcb2ffafc82cccf4d8bbaba97             dingo
001cdf01b096e06d78e9e5112d419397          pekinese
00214f311d5d2247d5dfe4fe24b2303d          bluetick
0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import GlobalMaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [6]:
num_classes = len(targets.unique())
print(num_classes)

120


In [7]:
actual_labels = []

In [8]:
desired_image_size = (128,128)

In [9]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
from keras.preprocessing.image import img_to_array

mypath='train'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
train_images = numpy.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
    path = onlyfiles[n].split(".")[0]
    lab = label_df.get_value(path,0)
    actual_labels.append(lab)
    new = cv2.imread(join(mypath,onlyfiles[n]))
    new = cv2.resize(new,desired_image_size)
    train_images[n] = img_to_array(new)/255.0

In [10]:
actual_labels = np.array(actual_labels)

In [11]:
submission_indices = []

In [12]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
from keras.preprocessing.image import img_to_array

mypath='test'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
test_images = numpy.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
    path_test = onlyfiles[n].split(".")[0]
    submission_indices.append(path_test)
    new = cv2.imread(join(mypath,onlyfiles[n]))
    new = cv2.resize(new,desired_image_size)
    test_images[n] = img_to_array(new)/255.0

In [13]:
submission_indices = np.array(submission_indices)

In [14]:
from keras.preprocessing.image import ImageDataGenerator
adder = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                           zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [15]:
train_images = train_images
target_train = actual_labels
test_images = test_images

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
numeric_labels = le.fit_transform(target_train)

In [17]:
num_classes_sample = len(np.unique(numeric_labels))
from keras.utils import to_categorical
train_labels = to_categorical(numeric_labels,num_classes_sample)

In [4]:
#model = Sequential()
input_shape = (desired_image_size[0],desired_image_size[1],3)
#model.add(Conv2D(20,(5,5),padding = "same",activation = "relu",input_shape = input_shape))
#model.add(MaxPooling2D(pool_size = (2,2),strides = (2,2)))
#model.add(Conv2D(50,(5,5),padding = "same",activation = "relu"))
#model.add(MaxPooling2D(pool_size = (2,2),strides = (2,2)))
#model.add(Flatten())
#model.add(Dense(500,activation = "relu"))
#model.add(Dense(num_classes_sample,activation = "softmax"))


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = input_shape, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

In [5]:
model.compile(loss = "categorical_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 64)        36928     
__________

In [ ]:
(dim2,dim3,dim4) = train_images[0].shape
num_images = train_images.size
training_data = np.zeros((num_images,dim2,dim3,dim4))
for i in range(num_images):
    training_data[i,:,:,:] = train_images[i]

In [ ]:
#model.fit(training_data,train_labels,batch_size = 30, epochs = 100)
bs = 256
A = model.fit_generator(adder.flow(training_data, train_labels, batch_size=bs), 
                        steps_per_epoch=num_images // bs,epochs=5000)

In [ ]:
model.save("my_model.h5")

In [ ]:
(dim2,dim3,dim4) = test_images[0].shape
num_test_images = test_images.size
practice_test_images = np.zeros((num_test_images,dim2,dim3,dim4))
for i in range(num_test_images):
    practice_test_images[i,:,:,:] = test_images[i]

In [ ]:
predictions = model.predict(practice_test_images)
print(predictions.shape)
print(predictions[0])

In [ ]:
number_labels_back = np.argmax(predictions,axis = 1)
print(len(number_labels_back))

In [ ]:
print(number_labels_back[0])

In [ ]:
probabilities = np.amax(predictions,axis = 1)

In [ ]:
final_prediction_strings = le.inverse_transform(number_labels_back)
print(final_prediction_strings)

In [ ]:
for i in range(30):
    print("Test image %d is of a(n) %s with probability %f" %(i,final_prediction_strings[i],probabilities[i]))

In [ ]:
numbers = np.array([i for i in range(num_classes_sample)])
column_names = le.inverse_transform(numbers)
final_submission = pd.DataFrame(predictions,columns = column_names, index = submission_indices)

In [ ]:
final_submission.to_csv("dog_test_predictions.csv")

In [ ]:
print(final_submission.head())